# Daily Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
cta = pd.read_csv('../capstone-data/daily/CTA_-_Ridership_-_Daily_Boarding_Totals.csv')
weather = pd.read_csv('../capstone-data/daily/weather.csv')
gas = pd.read_excel('../capstone-data/daily/gasoline_stocks.xlsx')
covid20 = pd.read_csv('../capstone-data/covid/OxCGRT_withnotes_2020.csv')
covid21 = pd.read_csv('../capstone-data/covid/OxCGRT_withnotes_2021.csv')
vax = pd.read_csv('../capstone-data/covid/COVID-19_Daily_Vaccinations_-_Chicago_Residents.csv')
curve = pd.read_excel('../capstone-data/daily/treasury_yield_curve.xlsx')
crash = pd.read_csv('../capstone-data/monthly/Traffic_Crashes_-_Crashes.csv')

/opt/anaconda3/envs/dsi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,3,61,63,64,65,71) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/envs/dsi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,3,59,61,63,71) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## CTA Data

In [3]:
cta.tail()

,service_date,day_type,bus,rail_boardings,total_rides
7696,11/26/2021,W,257700,189694,447394
7697,11/27/2021,A,237839,187065,424904
7698,11/28/2021,U,184817,147830,332647
7699,11/29/2021,W,421322,276090,697412
7700,11/30/2021,W,450230,302349,752579


In [4]:
cta.drop(columns={'day_type'},inplace=True)

In [5]:
cta.shape

(7701, 4)

In [6]:
cta.rename(columns={'service_date':'date'},inplace=True)

In [7]:
cta['date'] = pd.to_datetime(cta['date'])

In [8]:
cta.set_index('date',inplace=True)

cta.sort_index(inplace=True)

In [9]:
cta.dtypes

bus               int64
rail_boardings    int64
total_rides       int64
dtype: object

In [10]:
cta.isna().sum()

bus               0
rail_boardings    0
total_rides       0
dtype: int64

## Weather Data

In [11]:
def daily_weather(weather):
    """
    A function that cleans the weather dataset, which comes as daily. 
    Commented out is optional code that can be edited to suit the needed format,
    ex. ...resample('MS').sum() for month starting on first day of that month.
    """
    weather.columns
    weather.columns = weather.columns.str.lower()
    weather.head()
    weather.dtypes
    weather.drop(columns='station',inplace=True)
    weather['date'] = pd.to_datetime(weather['date'])
    weather.set_index('date',inplace=True)
    weather.sort_index(inplace=True)
    weather = weather.fillna(0)
    
    
    # weather = df.resample('W',closed='left').sum()
    # weather.index = weather.index - pd.DateOffset(days=6)
    
    
    return weather


In [12]:
weather = daily_weather(weather)

In [13]:
weather.isna().sum()

prcp    0
snow    0
snwd    0
tmax    0
tmin    0
dtype: int64

In [14]:
cta.isna().sum()

bus               0
rail_boardings    0
total_rides       0
dtype: int64

In [15]:
cta.shape, weather.shape

((7701, 3), (7694, 5))

### Merge CTA/Weather

In [16]:
daily = cta.merge(weather, how='outer', left_index=True, right_index=True)
daily

,bus,rail_boardings,total_rides,prcp,snow,snwd,tmax,tmin
date,,,,,,,,
2001-01-01,297192.0,126455.0,423647.0,0.0,0.0,16.0,26,13
2001-01-02,780827.0,501952.0,1282779.0,0.0,0.0,15.0,21,7
2001-01-03,824923.0,536432.0,1361355.0,0.0,0.0,15.0,28,7
2001-01-04,870021.0,550011.0,1420032.0,0.0,0.0,14.0,32,23
2001-01-05,890426.0,557917.0,1448343.0,0.0,0.0,12.0,37,24
...,...,...,...,...,...,...,...,...
2022-01-20,NaN,NaN,NaN,0.0,0.0,0.0,22,9
2022-01-21,NaN,NaN,NaN,0.0,0.0,0.0,27,10
2022-01-22,NaN,NaN,NaN,0.0,0.2,0.0,36,20


In [17]:
daily.loc[daily['bus'].isna()==True].head()

,bus,rail_boardings,total_rides,prcp,snow,snwd,tmax,tmin
date,,,,,,,,
2021-12-01,NaN,NaN,NaN,0.0,0.0,0.0,48,35
2021-12-02,NaN,NaN,NaN,0.0,0.0,0.0,59,44
2021-12-03,NaN,NaN,NaN,0.0,0.0,0.0,51,39
2021-12-04,NaN,NaN,NaN,0.0,0.0,0.0,46,31
2021-12-05,NaN,NaN,NaN,0.0,0.0,0.0,43,35


In [18]:
dailydrop=daily.dropna()

## Gas Data

In [19]:
gas

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,2022-01-31,2.5599,2.5741,2.5388,2.5543,2.5543,22434
1,2022-01-28,2.5608,2.5798,2.5244,2.5423,2.5423,23546
2,2022-01-27,2.5223,2.57,2.51,2.521,2.521,23546
3,2022-01-26,2.4552,2.5347,2.4533,2.5229,2.5229,35505
4,2022-01-25,2.4215,2.465,2.4023,2.4595,2.4595,30337
...,...,...,...,...,...,...,...
5361,2001-01-08,0.827,0.853,0.825,0.835,0.835,24057
5362,2001-01-05,0.815,0.838,0.814,0.8205,0.8205,18617
5363,2001-01-04,0.815,0.83,0.813,0.8186,0.8186,17869
5364,2001-01-03,0.799,0.812,0.792,0.81,0.81,10219


In [20]:
gas.dtypes

Date           datetime64[ns]
Open                   object
High                   object
Low                    object
Close*                 object
Adj Close**            object
Volume                 object
dtype: object

In [21]:
gas.isna().sum()

Date           0
Open           0
High           0
Low            0
Close*         0
Adj Close**    0
Volume         0
dtype: int64

In [22]:
def clean_gas(gas):
    gas.columns = gas.columns.str.lower()
    gas['date'] = pd.to_datetime(gas['date'])
    gas.set_index('date',inplace=True)
    gas.sort_index(inplace=True)
    gas.drop(columns={'high','low','adj close**'},inplace=True)
    gas.replace('-',np.nan,inplace=True)
    gas.rename(columns={'open':'gas_open','close*': 'gas_close',
                        'volume':'gas_volume'},inplace=True)
    return gas
    

In [23]:
gas = clean_gas(gas)

In [24]:
gas.isna().sum()

gas_open       69
gas_close      69
gas_volume    142
dtype: int64

In [25]:
gas['gas_volume'].isna().value_counts(normalize=True)

False    0.973537
True     0.026463
Name: gas_volume, dtype: float64

In [26]:
gas.loc[gas['gas_volume'].isna()==True].head()

,gas_open,gas_close,gas_volume
date,,,
2001-01-15,NaN,NaN,NaN
2001-01-29,0.891,0.877,NaN
2001-02-19,NaN,NaN,NaN
2001-03-13,0.875,0.885,NaN
2001-04-12,NaN,NaN,NaN


In [27]:
gas.describe()

,gas_open,gas_close,gas_volume
count,5297.000000,5297.000000,5224.000000
mean,1.829898,1.830688,40378.104326
std,0.703186,0.703521,21033.533512
min,0.427300,0.411800,5.000000
25%,1.306600,1.305800,24325.000000
50%,1.763900,1.767600,36029.500000
75%,2.297500,2.300900,54194.750000
max,3.563000,3.571000,190777.000000


In [28]:
gas.tail()

,gas_open,gas_close,gas_volume
date,,,
2022-01-25,2.4215,2.4595,30337.0
2022-01-26,2.4552,2.5229,35505.0
2022-01-27,2.5223,2.5210,23546.0
2022-01-28,2.5608,2.5423,23546.0
2022-01-31,2.5599,2.5543,22434.0


### Merge Gasoline

In [29]:
daily.shape

(7756, 8)

In [30]:
dailydrop.shape

(7701, 8)

In [31]:
gas.shape

(5366, 3)

In [32]:
gas.isna().sum()

gas_open       69
gas_close      69
gas_volume    142
dtype: int64

In [33]:
dailydrop = dailydrop.merge(gas, how='outer', left_index=True, right_index=True)


In [34]:
dailydrop.isna().sum()

bus                 42
rail_boardings      42
total_rides         42
prcp                42
snow                42
snwd                42
tmax                42
tmin                42
gas_open          2403
gas_close         2403
gas_volume        2476
dtype: int64

In [35]:
dailydrop.drop(dailydrop.tail(42).index,inplace=True)
dailydrop.tail()

,bus,rail_boardings,total_rides,prcp,snow,snwd,tmax,tmin,gas_open,gas_close,gas_volume
date,,,,,,,,,,,
2021-11-26,257700.0,189694.0,447394.0,0.0,0.0,0.0,31.0,19.0,2.3140,2.0294,17599.0
2021-11-27,237839.0,187065.0,424904.0,0.0,0.0,0.0,41.0,30.0,NaN,NaN,NaN
2021-11-28,184817.0,147830.0,332647.0,0.0,0.0,0.0,41.0,26.0,NaN,NaN,NaN
2021-11-29,421322.0,276090.0,697412.0,0.0,0.0,0.0,41.0,26.0,2.0785,2.0771,29118.0
2021-11-30,450230.0,302349.0,752579.0,0.0,0.0,0.0,53.0,33.0,2.1036,1.9801,83219.0


## Treasury Curve

In [36]:
curve.dtypes

Date     datetime64[ns]
1 Mo            float64
2 Mo            float64
3 Mo            float64
6 Mo            float64
1 Yr            float64
2 Yr            float64
3 Yr            float64
5 Yr            float64
7 Yr            float64
10 Yr           float64
20 Yr           float64
30 Yr           float64
dtype: object

In [37]:
curve.shape

(5274, 13)

In [38]:
curve.isna().sum()

Date        0
1 Mo      145
2 Mo     4451
3 Mo        3
6 Mo        0
1 Yr        0
2 Yr        0
3 Yr        0
5 Yr        0
7 Yr        0
10 Yr       0
20 Yr       0
30 Yr     994
dtype: int64

In [39]:
def clean_curve(curve):
    curve.columns = curve.columns.str.lower()

    curve.set_index('date',inplace=True)

    curve.sort_index(inplace=True)
    curve.columns = curve.columns.str.replace(' ','_')
    curve.drop(columns={'1_mo','2_mo','30_yr'},inplace=True)
    return curve

In [40]:
curve= clean_curve(curve)

In [41]:
curve.head()

,3_mo,6_mo,1_yr,2_yr,3_yr,5_yr,7_yr,10_yr,20_yr
date,,,,,,,,,
2001-01-02,5.87,5.58,5.11,4.87,4.82,4.76,4.97,4.92,5.46
2001-01-03,5.69,5.44,5.04,4.92,4.92,4.94,5.18,5.14,5.62
2001-01-04,5.37,5.20,4.82,4.77,4.78,4.82,5.07,5.03,5.56
2001-01-05,5.12,4.98,4.60,4.56,4.57,4.66,4.93,4.93,5.50
2001-01-08,5.19,5.03,4.61,4.54,4.55,4.65,4.94,4.94,5.52


### Merge Yield Curve

In [42]:
dailydrop = dailydrop.merge(curve, how='outer', left_index=True, right_index=True)


In [43]:
dailydrop.isna().sum()[0]

42

In [44]:
dailydrop = dailydrop.drop(dailydrop.tail(42).index)

In [45]:
dailydrop.tail()

,bus,rail_boardings,total_rides,prcp,snow,snwd,tmax,tmin,gas_open,gas_close,gas_volume,3_mo,6_mo,1_yr,2_yr,3_yr,5_yr,7_yr,10_yr,20_yr
date,,,,,,,,,,,,,,,,,,,,
2021-11-26,257700.0,189694.0,447394.0,0.0,0.0,0.0,31.0,19.0,2.3140,2.0294,17599.0,0.06,0.1,0.20,0.50,0.81,1.16,1.40,1.48,1.89
2021-11-27,237839.0,187065.0,424904.0,0.0,0.0,0.0,41.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-28,184817.0,147830.0,332647.0,0.0,0.0,0.0,41.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-29,421322.0,276090.0,697412.0,0.0,0.0,0.0,41.0,26.0,2.0785,2.0771,29118.0,0.06,0.1,0.21,0.51,0.83,1.18,1.42,1.52,1.93
2021-11-30,450230.0,302349.0,752579.0,0.0,0.0,0.0,53.0,33.0,2.1036,1.9801,83219.0,0.05,0.1,0.24,0.52,0.81,1.14,1.36,1.43,1.85


In [46]:
dailydrop.isna().sum()

bus                  0
rail_boardings       0
total_rides          0
prcp                 0
snow                 0
snwd                 0
tmax                 0
tmin                 0
gas_open          2403
gas_close         2403
gas_volume        2476
3_mo              2430
6_mo              2427
1_yr              2427
2_yr              2427
3_yr              2427
5_yr              2427
7_yr              2427
10_yr             2427
20_yr             2427
dtype: int64

## Prep Covid

In [47]:
def merge_prep_covid(df):

    df = df.loc[(df['CountryCode']=='USA') & (df['RegionName']=='Illinois')]

    test20 = df.drop(columns={'CountryName','CountryCode','RegionName','RegionCode','Jurisdiction','H1_Flag',
                        'StringencyIndexForDisplay','StringencyLegacyIndexForDisplay','GovernmentResponseIndexForDisplay',
                        'ContainmentHealthIndexForDisplay','EconomicSupportIndexForDisplay',
                        'C7_Flag','E1_Flag','H6_Flag','H7_Flag','H8_Flag',
                        'V2B_Vaccine age eligibility/availability age floor (general population summary)',
                       'V2D_Medically/ clinically vulnerable (Non-elderly)', 'V2E_Education',
                       'V2F_Frontline workers  (non healthcare)',
                       'V2G_Frontline workers  (healthcare)','C1_Flag','C1_Notes','C2_Flag','C2_Notes',
                             'C3_Flag','C3_Notes'})
    
    test20 = test20.drop(test20.columns[test20.isnull().sum() > 30],axis=1)
    
    #test20['Date'] = pd.to_datetime(test20['Date'],format='%Y%m%d').dt.date
    test20['Date'] = pd.to_datetime(test20['Date'],format='%Y%m%d')
    # test20 = test20.groupby('Date').agg('mean')
    test20.set_index('Date',inplace=True)
    test20.sort_index(inplace=True)
    #test20=test20.groupby(pd.Grouper(freq='M')).nth(0)
    
    return test20

In [48]:
covid20.head()

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,C2_Flag,C2_Notes,C3_Cancel public events,C3_Flag,C3_Notes,C4_Restrictions on gatherings,C4_Flag,C4_Notes,C5_Close public transport,C5_Flag,C5_Notes,C6_Stay at home requirements,C6_Flag,C6_Notes,C7_Restrictions on internal movement,C7_Flag,C7_Notes,C8_International travel controls,C8_Notes,E1_Income support,E1_Flag,E1_Notes,E2_Debt/contract relief,E2_Notes,E3_Fiscal measures,E3_Notes,E4_International support,E4_Notes,H1_Public information campaigns,H1_Flag,H1_Notes,H2_Testing policy,H2_Notes,H3_Contact tracing,H3_Notes,H4_Emergency investment in healthcare,H4_Notes,H5_Investment in vaccines,H5_Notes,H6_Facial Coverings,H6_Flag,H6_Notes,H7_Vaccination policy,H7_Flag,H7_Notes,H8_Protection of elderly people,H8_Flag,H8_Notes,M1_Wildcard,M1_Notes,V1_Vaccine Prioritisation (summary),V1_Notes,V2A_Vaccine Availability (summary),V2_Notes,V2B_Vaccine age eligibility/availability age floor (general population summary),V2C_Vaccine age eligibility/availability age floor (at risk summary),V2D_Medically/ clinically vulnerable (Non-elderly),V2E_Education,V2F_Frontline workers (non healthcare),V2G_Frontline workers (healthcare),V3_Vaccine Financial Support (summary),V3_Notes,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200101,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200102,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200103,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200104,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200105,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
covid20 = merge_prep_covid(covid20)

In [50]:
covid20.isna().sum()

C1_School closing                        0
C2_Workplace closing                     0
C3_Cancel public events                  0
C4_Restrictions on gatherings            0
C5_Close public transport                0
C6_Stay at home requirements             0
C7_Restrictions on internal movement     0
C8_International travel controls         0
E1_Income support                        0
E2_Debt/contract relief                  0
H1_Public information campaigns          0
H2_Testing policy                        0
H3_Contact tracing                       0
H6_Facial Coverings                      0
H7_Vaccination policy                    0
H8_Protection of elderly people          0
ConfirmedCases                          21
ConfirmedDeaths                         21
StringencyIndex                          0
StringencyLegacyIndex                    0
GovernmentResponseIndex                  0
ContainmentHealthIndex                   0
EconomicSupportIndex                     0
dtype: int6

In [51]:
covid21 = merge_prep_covid(covid21)

In [52]:
covid_total = pd.concat([covid20, covid21], axis=0)

In [53]:
covid_total.loc[covid_total['ConfirmedCases'].isna()==True]

,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,E1_Income support,E2_Debt/contract relief,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings,H7_Vaccination policy,H8_Protection of elderly people,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyLegacyIndex,GovernmentResponseIndex,ContainmentHealthIndex,EconomicSupportIndex
Date,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.0
2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.0
2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.0
2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.0
2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.0
2020-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.0
2020-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.0
2020-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.0
2020-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.00,0.00,0.0


In [54]:
covid_total=covid_total.fillna(0.0)

In [89]:
#covid_total.to_csv('../capstone-data/daily/covid_total(d).csv')

### Merge Covid Restrictions

In [55]:
dailydrop = dailydrop.merge(covid_total, how='outer', left_index=True, right_index=True)
dailydrop

,bus,rail_boardings,total_rides,prcp,snow,snwd,tmax,tmin,gas_open,gas_close,gas_volume,3_mo,6_mo,1_yr,2_yr,3_yr,5_yr,7_yr,10_yr,20_yr,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,E1_Income support,E2_Debt/contract relief,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings,H7_Vaccination policy,H8_Protection of elderly people,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyLegacyIndex,GovernmentResponseIndex,ContainmentHealthIndex,EconomicSupportIndex
2001-01-01,297192.0,126455.0,423647.0,0.0,0.0,16.0,26.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-02,780827.0,501952.0,1282779.0,0.0,0.0,15.0,21.0,7.0,0.794,0.7950,14697.0,5.87,5.58,5.11,4.87,4.82,4.76,4.97,4.92,5.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,824923.0,536432.0,1361355.0,0.0,0.0,15.0,28.0,7.0,0.799,0.8100,10219.0,5.69,5.44,5.04,4.92,4.92,4.94,5.18,5.14,5.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-04,870021.0,550011.0,1420032.0,0.0,0.0,14.0,32.0,23.0,0.815,0.8186,17869.0,5.37,5.20,4.82,4.77,4.78,4.82,5.07,5.03,5.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,890426.0,557917.0,1448343.0,0.0,0.0,12.0,37.0,24.0,0.815,0.8205,18617.0,5.12,4.98,4.60,4.56,4.57,4.66,4.93,4.93,5.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,2.0,3.0,2.0,2.0,5.0,2.0,2077260.0,30021.0,25.0,34.52,44.79,45.83,37.5
2021-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,2.0,3.0,2.0,2.0,5.0,2.0,2098064.0,30117.0,25.0,34.52,44.79,45.83,37.5
2021-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,2.0,3.0,2.0,2.0,5.0,2.0,2119162.0,30167.0,25.0,34.52,44.79,45.83,37.5
2021-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,2.0,3.0,2.0,2.0,5.0,2.0,2149548.0,30254.0,25.0,34.52,44.79,45.83,37.5


In [56]:
dailydrop.isna().sum()[0]

31

In [57]:
dailydrop = dailydrop.drop(dailydrop.tail(31).index)

In [58]:
dailydrop.iloc[:,20:43].fillna(0.0,inplace=True)

/opt/anaconda3/envs/dsi/lib/python3.8/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [59]:
dailydrop.isna().sum()

bus                                        0
rail_boardings                             0
total_rides                                0
prcp                                       0
snow                                       0
snwd                                       0
tmax                                       0
tmin                                       0
gas_open                                2403
gas_close                               2403
gas_volume                              2476
3_mo                                    2430
6_mo                                    2427
1_yr                                    2427
2_yr                                    2427
3_yr                                    2427
5_yr                                    2427
7_yr                                    2427
10_yr                                   2427
20_yr                                   2427
C1_School closing                          0
C2_Workplace closing                       0
C3_Cancel 

## Prep Vax

In [60]:
def vax_cleaning(vax):
    vax.columns = vax.columns.str.lower()
    vax['date'] = pd.to_datetime(vax['date'])
    vax.set_index('date',inplace=True)
    vax.sort_index(inplace=True)
    
    vax=vax[['total doses - daily','total doses - cumulative','1st dose - daily',
             '1st dose - cumulative','1st dose - percent population','vaccine series completed - daily',
             'vaccine series completed - cumulative','vaccine series completed - percent population']]
    
    vax = vax.rename(columns={'total doses - daily':'total_daily_doses',
           'total doses - cumulative':'total_daily_cum',
           '1st dose - daily':'first_dose_daily',
           '1st dose - cumulative':'first_dose_cum',
           '1st dose - percent population':'first_dose_percent_pop',
           'vaccine series completed - daily':'vax_series_completed_daily',
           'vaccine series completed - cumulative':'vax_series_cum',
           'vaccine series completed - percent population':'vax_series_percent'})
    return vax


In [61]:
vax = vax_cleaning(vax)

In [62]:
vax.isna().sum()

total_daily_doses             0
total_daily_cum               0
first_dose_daily              0
first_dose_cum                0
first_dose_percent_pop        0
vax_series_completed_daily    0
vax_series_cum                0
vax_series_percent            0
dtype: int64

In [63]:
vax

,total_daily_doses,total_daily_cum,first_dose_daily,first_dose_cum,first_dose_percent_pop,vax_series_completed_daily,vax_series_cum,vax_series_percent
date,,,,,,,,
2020-12-15,17,17,17,17,0.000,0,0,0.000
2020-12-16,169,186,169,186,0.000,0,0,0.000
2020-12-17,1989,2175,1989,2175,0.001,0,0,0.000
2020-12-18,5729,7904,5729,7904,0.003,0,0,0.000
2020-12-19,3513,11417,3513,11417,0.004,0,0,0.000
...,...,...,...,...,...,...,...,...
2022-01-09,4623,4277770,1260,1963754,0.729,880,1757428,0.652
2022-01-10,11610,4289380,3705,1967459,0.730,1919,1759347,0.653
2022-01-11,13429,4302809,5114,1972573,0.732,2274,1761621,0.654


### Merge Vax

In [64]:
dailydrop = dailydrop.merge(vax, how='outer', left_index=True, right_index=True)
dailydrop

,bus,rail_boardings,total_rides,prcp,snow,snwd,tmax,tmin,gas_open,gas_close,gas_volume,3_mo,6_mo,1_yr,2_yr,3_yr,5_yr,7_yr,10_yr,20_yr,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,E1_Income support,E2_Debt/contract relief,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings,H7_Vaccination policy,H8_Protection of elderly people,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyLegacyIndex,GovernmentResponseIndex,ContainmentHealthIndex,EconomicSupportIndex,total_daily_doses,total_daily_cum,first_dose_daily,first_dose_cum,first_dose_percent_pop,vax_series_completed_daily,vax_series_cum,vax_series_percent
2001-01-01,297192.0,126455.0,423647.0,0.0,0.0,16.0,26.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-02,780827.0,501952.0,1282779.0,0.0,0.0,15.0,21.0,7.0,0.794,0.7950,14697.0,5.87,5.58,5.11,4.87,4.82,4.76,4.97,4.92,5.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,824923.0,536432.0,1361355.0,0.0,0.0,15.0,28.0,7.0,0.799,0.8100,10219.0,5.69,5.44,5.04,4.92,4.92,4.94,5.18,5.14,5.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-04,870021.0,550011.0,1420032.0,0.0,0.0,14.0,32.0,23.0,0.815,0.8186,17869.0,5.37,5.20,4.82,4.77,4.78,4.82,5.07,5.03,5.56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,890426.0,557917.0,1448343.0,0.0,0.0,12.0,37.0,24.0,0.815,0.8205,18617.0,5.12,4.98,4.60,4.56,4.57,4.66,4.93,4.93,5.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4623.0,4277770.0,1260.0,1963754.0,0.729,880.0,1757428.0,0.652
2022-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11610.0,4289380.0,3705.0,1967459.0,0.730,1919.0,1759347.0,0.653
2022-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13429.0,4302809.0,5114.0,1972573.0,0.732,2274.0,1761621.0,0.654
2022-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13091.0,4315900.0,4857.0,1977430.0,0.734,2305.0,1763926.0,0.655


In [65]:
dailydrop.isna().sum()[0]

44

In [66]:
dailydrop = dailydrop.drop(dailydrop.tail(44).index)

In [67]:
dailydrop.loc[dailydrop['total_daily_doses'].isna()==True]

,bus,rail_boardings,total_rides,prcp,snow,snwd,tmax,tmin,gas_open,gas_close,gas_volume,3_mo,6_mo,1_yr,2_yr,3_yr,5_yr,7_yr,10_yr,20_yr,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,E1_Income support,E2_Debt/contract relief,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings,H7_Vaccination policy,H8_Protection of elderly people,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyLegacyIndex,GovernmentResponseIndex,ContainmentHealthIndex,EconomicSupportIndex,total_daily_doses,total_daily_cum,first_dose_daily,first_dose_cum,first_dose_percent_pop,vax_series_completed_daily,vax_series_cum,vax_series_percent
2001-01-01,297192.0,126455.0,423647.0,0.00,0.0,16.0,26.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-02,780827.0,501952.0,1282779.0,0.00,0.0,15.0,21.0,7.0,0.7940,0.7950,14697.0,5.87,5.58,5.11,4.87,4.82,4.76,4.97,4.92,5.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,824923.0,536432.0,1361355.0,0.00,0.0,15.0,28.0,7.0,0.7990,0.8100,10219.0,5.69,5.44,5.04,4.92,4.92,4.94,5.18,5.14,5.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-04,870021.0,550011.0,1420032.0,0.00,0.0,14.0,32.0,23.0,0.8150,0.8186,17869.0,5.37,5.20,4.82,4.77,4.78,4.82,5.07,5.03,5.56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,890426.0,557917.0,1448343.0,0.00,0.0,12.0,37.0,24.0,0.8150,0.8205,18617.0,5.12,4.98,4.60,4.56,4.57,4.66,4.93,4.93,5.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-10,297368.0,144459.0,441827.0,0.00,0.0,0.0,57.0,31.0,1.2826,1.3166,71548.0,0.08,0.09,0.10,0.14,0.20,0.39,0.65,0.92,1.44,2.0,1.0,2.0,4.0,0.0,2.0,2.0,3.0,1.0,2.0,2.0,3.0,1.0,3.0,0.0,1.0,823531.0,14845.0,66.67,64.29,61.46,61.31,62.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-11,277727.0,138386.0,416113.0,0.85,0.0,0.0,45.0,35.0,1.3166,1.3077,61285.0,0.08,0.08,0.10,0.11,0.18,0.37,0.63,0.90,1.42,2.0,1.0,2.0,4.0,0.0,2.0,2.0,3.0,1.0,2.0,2.0,3.0,1.0,3.0,0.0,1.0,832951.0,15067.0,66.67,64.29,61.46,61.31,62.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-12,181545.0,92521.0,274066.0,1.04,0.0,0.0,42.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,4.0,0.0,2.0,2.0,3.0,1.0,2.0,2.0,3.0,1.0,3.0,0.0,1.0,841688.0,15232.0,66.67,64.29,61.46,61.31,62.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-13,150344.0,78380.0,228724.0,0.00,0.0,0.0,37.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,4.0,0.0,2.0,2.0,3.0,1.0,2.0,2.0,3.0,1.0,3.0,0.0,1.0,848904.0,15339.0,66.67,64.29,61.46,61.31,62.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
dailydrop.iloc[:,43:51].fillna(0.0,inplace=True)

/opt/anaconda3/envs/dsi/lib/python3.8/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [69]:
dailydrop.isna().sum()

bus                                        0
rail_boardings                             0
total_rides                                0
prcp                                       0
snow                                       0
snwd                                       0
tmax                                       0
tmin                                       0
gas_open                                2403
gas_close                               2403
gas_volume                              2476
3_mo                                    2430
6_mo                                    2427
1_yr                                    2427
2_yr                                    2427
3_yr                                    2427
5_yr                                    2427
7_yr                                    2427
10_yr                                   2427
20_yr                                   2427
C1_School closing                          0
C2_Workplace closing                       0
C3_Cancel 

## Prep Crash

In [70]:
crash = crash[['CRASH_DATE','DAMAGE']].copy()

In [71]:
crash['crash_occurrences'] = 1

In [72]:
crash['CRASH_DATE'] = pd.to_datetime(crash['CRASH_DATE']).dt.date
crash['CRASH_DATE'] = pd.to_datetime(crash['CRASH_DATE'])

In [73]:
crash.columns = crash.columns.str.lower()

In [74]:
crash['damage'].value_counts()

OVER $1,500      343278
$501 - $1,500    164722
$500 OR LESS      71227
Name: damage, dtype: int64

In [75]:
crash['damage'] = crash['damage'].replace('OVER $1,500', 3)
crash['damage'] = crash['damage'].replace('$501 - $1,500', 2)
crash['damage'] = crash['damage'].replace('$500 OR LESS', 1)

In [76]:
crash.rename(columns= {'crash_date':'date','damage':'damage_indicator'},inplace=True)

In [77]:
crash = crash.groupby('date').agg({'crash_occurrences':'sum','damage_indicator':'mean'})

In [78]:
crash.sort_index(inplace=True)

In [79]:
crash.isna().sum()

crash_occurrences    0
damage_indicator     0
dtype: int64

In [80]:
crash

,crash_occurrences,damage_indicator
date,,
2013-03-03,1,3.000000
2014-01-18,1,3.000000
2014-01-21,1,3.000000
2014-02-24,1,2.000000
2014-06-25,1,2.000000
...,...,...
2022-01-13,216,2.601852
2022-01-14,264,2.625000
2022-01-15,265,2.686792


### Merge Crash

In [81]:
dailydrop = dailydrop.merge(crash, how='outer', left_index=True, right_index=True)
dailydrop

,bus,rail_boardings,total_rides,prcp,snow,snwd,tmax,tmin,gas_open,gas_close,gas_volume,3_mo,6_mo,1_yr,2_yr,3_yr,5_yr,7_yr,10_yr,20_yr,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,E1_Income support,E2_Debt/contract relief,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings,H7_Vaccination policy,H8_Protection of elderly people,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyLegacyIndex,GovernmentResponseIndex,ContainmentHealthIndex,EconomicSupportIndex,total_daily_doses,total_daily_cum,first_dose_daily,first_dose_cum,first_dose_percent_pop,vax_series_completed_daily,vax_series_cum,vax_series_percent,crash_occurrences,damage_indicator
2001-01-01,297192.0,126455.0,423647.0,0.0,0.0,16.0,26.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2001-01-02,780827.0,501952.0,1282779.0,0.0,0.0,15.0,21.0,7.0,0.794,0.7950,14697.0,5.87,5.58,5.11,4.87,4.82,4.76,4.97,4.92,5.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2001-01-03,824923.0,536432.0,1361355.0,0.0,0.0,15.0,28.0,7.0,0.799,0.8100,10219.0,5.69,5.44,5.04,4.92,4.92,4.94,5.18,5.14,5.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2001-01-04,870021.0,550011.0,1420032.0,0.0,0.0,14.0,32.0,23.0,0.815,0.8186,17869.0,5.37,5.20,4.82,4.77,4.78,4.82,5.07,5.03,5.56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2001-01-05,890426.0,557917.0,1448343.0,0.0,0.0,12.0,37.0,24.0,0.815,0.8205,18617.0,5.12,4.98,4.60,4.56,4.57,4.66,4.93,4.93,5.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216.0,2.601852
2022-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264.0,2.625000
2022-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265.0,2.686792
2022-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.0,2.697987


In [82]:
dailydrop.isna().sum()[0]

48

In [83]:
dailydrop = dailydrop.drop(dailydrop.tail(48).index)

In [84]:
dailydrop.isna().sum()

bus                                        0
rail_boardings                             0
total_rides                                0
prcp                                       0
snow                                       0
snwd                                       0
tmax                                       0
tmin                                       0
gas_open                                2403
gas_close                               2403
gas_volume                              2476
3_mo                                    2430
6_mo                                    2427
1_yr                                    2427
2_yr                                    2427
3_yr                                    2427
5_yr                                    2427
7_yr                                    2427
10_yr                                   2427
20_yr                                   2427
C1_School closing                          0
C2_Workplace closing                       0
C3_Cancel 

In [85]:
dailydrop['damage_indicator'] = dailydrop['damage_indicator'].fillna(0.0)
dailydrop['crash_occruences'] = dailydrop['crash_occurrences'].fillna(0.0)

/var/folders/nw/r1vxxfhd0f3gvq3n40jpl2980000gn/T/ipykernel_95681/3818812340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dailydrop['damage_indicator'] = dailydrop['damage_indicator'].fillna(0.0)
/var/folders/nw/r1vxxfhd0f3gvq3n40jpl2980000gn/T/ipykernel_95681/3818812340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dailydrop['crash_occruences'] = dailydrop['crash_occurrences'].fillna(0.0)


In [86]:
dailydrop.to_csv('../capstone-data/daily-merged.csv')